In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_key import key
from geoapi import geo_apikey

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy



In [6]:
#https://api.census.gov/data/2019/acs/acs5?get=NAME,group(B01003)&for=city:*&key=YOUR_API_KEY

import requests
import json
# Set base URL
base_url = "https://api.census.gov/data"
year = '/2022'
data_name = '/acs/acs1/subject'
#all of the B02015 table ID data for all states
#qry = '?get=group(S1903)&for=place'
qry = "?get=NAME,S1903_C03_001E&for=place"
key = str('&key='+ key)

    # Make an API request using the params dictionary
test = str(base_url + year + data_name + qry + key)
test
response = requests.get(base_url + year + data_name + qry + key)
result = response.json()
print (json.dumps(result, indent=4))


[
    [
        "NAME",
        "S1903_C03_001E",
        "state",
        "place"
    ],
    [
        "Auburn city, Alabama",
        "54839",
        "01",
        "03076"
    ],
    [
        "Birmingham city, Alabama",
        "39326",
        "01",
        "07000"
    ],
    [
        "Dothan city, Alabama",
        "53929",
        "01",
        "21184"
    ],
    [
        "Hoover city, Alabama",
        "103194",
        "01",
        "35896"
    ],
    [
        "Huntsville city, Alabama",
        "68930",
        "01",
        "37000"
    ],
    [
        "Mobile city, Alabama",
        "50051",
        "01",
        "50000"
    ],
    [
        "Montgomery city, Alabama",
        "55108",
        "01",
        "51000"
    ],
    [
        "Tuscaloosa city, Alabama",
        "49005",
        "01",
        "77256"
    ],
    [
        "Anchorage municipality, Alaska",
        "100751",
        "02",
        "03000"
    ],
    [
        "Avondale city, Arizona",
        "66801

In [7]:
#https://www.pgpf.org/blog/2023/11/income-and-wealth-in-the-united-states-an-overview-of-recent-data#:~:text=According%20to%20the%20ASEC%2C%20median,the%20pandemic%20and%20subsequent%20recession.
median_income = 74580
# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Convert JSON data to DataFrame
    df = pd.DataFrame(data[1:], columns=data[0:])
    
    # Save the DataFrame to a CSV file
    df.to_csv("../data/census_data.csv", index=False)
    print("Data saved to census_data.csv")
else:
    print("Error:", response.status_code)

Data saved to census_data.csv


In [21]:
# Convert the cities weather data into a Pandas DataFrame
city_eco_data_df  = pd.read_csv('../data/census_data.csv')

# Splitting the 'NAME' column into two columns 'City' and 'State'
city_eco_data_df[['City', 'State']] = city_eco_data_df['NAME'].str.split(', ', expand=True)
#city_eco_data_df.drop(columns=['NAME'], inplace=True)

# Show Record Count
city_eco_data_df.count()
city_eco_data_df.head()

,NAME,S1903_C03_001E,state,place,City,State
0,"Auburn city, Alabama",54839,1,3076,Auburn city,Alabama
1,"Birmingham city, Alabama",39326,1,7000,Birmingham city,Alabama
2,"Dothan city, Alabama",53929,1,21184,Dothan city,Alabama
3,"Hoover city, Alabama",103194,1,35896,Hoover city,Alabama
4,"Huntsville city, Alabama",68930,1,37000,Huntsville city,Alabama


In [22]:
## splitting the eco median data into low income and high income..
city_eco_data_df = pd.DataFrame(city_eco_data_df)

# Splitting the DataFrame into two based on the condition
threshold_value = 74580
df_above_threshold = city_eco_data_df[city_eco_data_df['S1903_C03_001E'] > threshold_value]
df_below_threshold = city_eco_data_df[city_eco_data_df['S1903_C03_001E'] <= threshold_value]

df_above_threshold, df_below_threshold


(                                NAME  S1903_C03_001E  state  place  \
 3               Hoover city, Alabama          103194      1  35896   
 8     Anchorage municipality, Alaska          100751      2   3000   
 10             Buckeye city, Arizona           95235      4   7940   
 11         Casas Adobes CDP, Arizona           75650      4  10670   
 12            Chandler city, Arizona           98664      4  12000   
 ...                              ...             ...    ...    ...   
 1269      South Hill CDP, Washington          109848     53  65922   
 1272         Tacoma city, Washington           80784     53  70000   
 1273      Vancouver city, Washington           76799     53  74060   
 1275        Appleton city, Wisconsin           75262     55   2375   
 1284        Waukesha city, Wisconsin           74675     55  84250   
 
                         City       State  
 3                Hoover city     Alabama  
 8     Anchorage municipality      Alaska  
 10           

In [23]:
def get_coordinates(city_name, geo_apikey):
    base_url = "https://api.geoapify.com/v1/geocode/search"
    params = {
        "text": city_name,
        "apiKey": geo_apikey
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            # Extracting the first result's coordinates
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
        else:
            return "No results found"
    else:
        return "Error in API request"

# pull the data
api_key = geo_apikey # Replace with your actual Geoapify API key
city_name1 = "New York"
coordinates = get_coordinates(city_name1, api_key)
print(f"Coordinates of {city_name1}: {coordinates}")

Coordinates of New York: [-74.0060152, 40.7127281]


In [24]:

# Empty DataFrame to store the results
city_coordinates_df = pd.DataFrame(columns=['City', 'Latitude', 'Longitude'])

# Iterating through the cities and fetching coordinates
for city in city_eco_data_df['NAME']:
    coordinates = get_coordinates(city, api_key)
    if coordinates != [None, None]:
        # Creating a temporary DataFrame for the current city
        temp_df = pd.DataFrame([{
            'City': city,
            'Latitude': coordinates[1],  # Latitude
            'Longitude': coordinates[0]  # Longitude
        }])
        # Concatenating the temporary DataFrame with the main DataFrame
        city_coordinates_df = pd.concat([city_coordinates_df, temp_df], ignore_index=True)
        print(f"getting data for {city}")
    else:
        print(f"Failed to fetch coordinates for {city}")

# Display the DataFrame
print(city_coordinates_df.head())

getting data for Auburn city, Alabama
getting data for Birmingham city, Alabama
getting data for Dothan city, Alabama


KeyboardInterrupt: 

In [ ]:
# Join the DataFrames on the city names
merged_df = city_eco_data_df.merge(geoapify_df, left_on='NAME', right_on='City')

# Splitting the merged DataFrame into two based on the condition
threshold_value = 74580
df_above_threshold = merged_df[merged_df['S1903_C03_001E'] > threshold_value]
df_below_threshold = merged_df[merged_df['S1903_C03_001E'] <= threshold_value]

df_above_threshold, df_below_threshold

In [ ]:
#PLACES Search with coords from geocode and census data DFs..       
##still in progess

radius = 16093.4  # 10 miles in meters

# Initialize an empty DataFrame
df = pd.DataFrame(columns=["City", "Commercial Weapons Count"])

# Iterate through each city
for city, lat, lon in cities:
    params = {
        "apiKey": geoapify_key,
        "categories": "commercial.weapon",
        "filter": f"circle:{lon},{lat},{radius}",
        "limit": 1  # Adjust as needed
    }
    
    response = requests.get("https://api.geoapify.com/v2/places", params=params)
    
    if response.status_code == 200:
        data = response.json()
        count = len(data['features'])  # Count of commercial weapons
        df = df.append({"City": city, "Commercial Weapons Count": count}, ignore_index=True)
    else:
        print(f"Error fetching data for {city}")

# Save the DataFrame to a CSV file
df.to_csv("city_commercial_weapons.csv", index=False)

print("Data saved to city_commercial_weapons.csv")
